In [1]:
import pandas as pd
import numpy as np

from pyomo.environ import *

from data_cleaning import site_occ_dict, blockgroup_pop_dict, bg_ces_df, dist_to_site_df

#dist_to_site_df = pd.read_csv('data/distmatrix_contracosta.csv')
#dist_to_site_df.set_index('Unnamed: 0', inplace = True)
#dist_to_site_df.index.name = None

### Define Model
_Create model and initial indices_

In [2]:
model = ConcreteModel()

model.init_bgs = Set(initialize = dist_to_site_df.index)
model.init_sites = Set(initialize = dist_to_site_df.columns)

_Create set of blockgroup, site pairs within five miles of each other_

In [3]:
def filter_to_nearby_sites(model, bg, site):
    return not np.isnan(dist_to_site_df.loc[bg, site])

model.idx_bg_site_pairs = Set(initialize = model.init_bgs*model.init_sites, filter = filter_to_nearby_sites)

_Create new index of sites and bgs only including those that belong in >5 mile pair_

In [4]:
filtered_bgs = []
filtered_sites = []

for pair in model.idx_bg_site_pairs:
    if pair[0] not in filtered_bgs:
        filtered_bgs.append(pair[0])
    if pair[1] not in filtered_sites:
        filtered_sites.append(pair[1])
        
model.idx_bgs = Set(initialize = filtered_bgs)
model.idx_sites = Set(initialize = filtered_sites)

### Define Parameters

_Number of people per blockgroup_

In [5]:
def get_blockgroup_pop(model, bg):
    return(blockgroup_pop_dict[bg])

model.param_bg_pop = Param(model.idx_bgs, initialize = get_blockgroup_pop)

_Distance from blockgroup to site_

In [6]:
def get_bg_site_dist(model, bg, site):
    return(dist_to_site_df.loc[bg, site])

model.param_bg_site_dist = Param(model.idx_bg_site_pairs, initialize = get_bg_site_dist)

_Max capacity per site_

In [7]:
def get_site_capacity(model, site):
    return(site_occ_dict[site])

model.param_site_cap = Param(model.idx_sites, initialize = get_site_capacity)

_Sites within range for each blockgroup_

In [8]:
bg_sites_in_range = {bg: [] for bg in model.idx_bgs}

for bg in bg_sites_in_range:
    for pair in model.idx_bg_site_pairs:
        if pair[0] == bg:
            bg_sites_in_range[bg].append(pair[1])

bg_with_no_hub = [key for key in bg_sites_in_range if len(bg_sites_in_range[key]) == 0]

for bg in bg_with_no_hub:
    del bg_sites_in_range[bg]
    
model.param_bg_sites_in_range = Param(model.idx_bgs, within=Any, initialize=bg_sites_in_range)

_Blockgroups within range for each site_

In [9]:
site_bgs_in_range = {site: [] for site in model.idx_sites}

for site in site_bgs_in_range:
    for pair in model.idx_bg_site_pairs:
        if pair[1] == site:
            site_bgs_in_range[site].append(pair[0])

site_with_no_hub = [key for key in site_bgs_in_range if len(site_bgs_in_range[key]) == 0]

for site in site_with_no_hub:
    del site_bgs_in_range[site]
    
model.param_site_bgs_in_range = Param(model.idx_sites, within=Any, initialize=site_bgs_in_range)

### Define Variables

_Is this site a site?_

In [10]:
model.var_hub_yn = Var(model.idx_sites, initialize = 1, within = Binary)

_What prop of this blockgroup is served at this site?_

In [11]:
model.var_prop_bg_at_site = Var(model.idx_bg_site_pairs, initialize = 1.0, bounds = (0.0, 1.0))

### Define Objective

In [12]:
agg_dist = sum(model.param_bg_pop[bg]*model.var_prop_bg_at_site[bg, site]*model.param_bg_site_dist[bg, site] 
               for bg, site in model.idx_bg_site_pairs)

model.obj_min_agg_dist = Objective(expr = agg_dist, sense = minimize)

### Define Constraints

_Construct set number of sites_

In [13]:
n_sites = sum(model.var_hub_yn[site] for site in model.idx_sites)

model.con_max_sites = Constraint(expr = (3, n_sites, 5))

_Do not let anyone go to a site if it is not a hub_

In [14]:
def serve_only_at_hubs(model, bg, site):
    return(model.var_prop_bg_at_site[bg, site] <= model.var_hub_yn[site])

model.con_serve_only_at_hubs = Constraint(model.idx_bg_site_pairs, rule = serve_only_at_hubs)

_Serve a minimum proportion of area population_

In [15]:
pop_area_tot = sum(model.param_bg_pop[bg] for bg in model.idx_bgs)
pop_service_tot = sum(model.param_bg_pop[bg]*model.var_prop_bg_at_site[bg, site] for bg, site in model.idx_bg_site_pairs)

model.con_min_service_pop = Constraint(expr = pop_service_tot >= 0.05*pop_area_tot)

_Do not serve more than 100% of a blockgroup's demand_

In [16]:
def serve_only_demand(model, bg):
    bg_tot_pop = model.param_bg_pop[bg]
    bg_tot_served = sum(model.param_bg_pop[bg]*model.var_prop_bg_at_site[bg, site] for site in model.param_bg_sites_in_range[bg])
    
    return((0, bg_tot_served, bg_tot_pop))

model.con_serve_only_demand = Constraint(model.idx_bgs, rule = serve_only_demand)

_Do not send more people to hub than it can fit_
* Debug note: when if statement written as ==1 (as opposed to !=1) and lowerbound added, (e.g. 0.75*site_tot_cap, site_tot_served, 1.0*site_tot_cap), serve_only_at_hubs stops working properly and many  hubs are added

In [17]:
def serve_max_capacity(model, site):
    site_tot_cap = model.param_site_cap[site]
    site_tot_served = sum(model.param_bg_pop[bg]*model.var_prop_bg_at_site[bg, site] for bg in model.param_site_bgs_in_range[site])
    
    if model.var_hub_yn[site].value != 1:
        return((0, site_tot_served, 0))
    else:      
        return((0, site_tot_served, site_tot_cap))

model.con_serve_max_capacity = Constraint(model.idx_sites, rule = serve_max_capacity)

### Set up Solver

In [18]:
import pyomo.opt as pyopt

In [19]:
pyopt.SolverFactory('glpk').solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 28472.3044600993, 'Upper bound': 28472.3044600993, 'Number of objectives': 1, 'Number of constraints': 25951, 'Number of variables': 24130, 'Number of nonzeros': 166114, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '79', 'Number of created subproblems': '79'}}, 'Error rc': 0, 'Time': 7.136845827102661}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [20]:
result_site = [site for site in model.idx_sites]
result_hub_yn = [model.var_hub_yn[site].value for site in model.idx_sites]

result_hub_yn_df = pd.DataFrame({'site_id': result_site, 'site_yn':result_hub_yn})

result_hub_yn_df = result_hub_yn_df.loc[result_hub_yn_df['site_yn'] > 0]

result_hub_yn_df

,site_id,site_yn
87,31273323,1.0
308,438203854,1.0
442,943344307,1.0
452,166435596,1.0
474,32935760,1.0


In [21]:
prop_bg_at_site_df = dist_to_site_df
prop_bg_at_site_df.loc[:] = np.NaN

for key in model.var_prop_bg_at_site.keys():
    prop_bg_at_site_df.loc[key] = model.var_prop_bg_at_site[key].value
    
prop_bg_at_site_df_notnull = prop_bg_at_site_df[prop_bg_at_site_df.notnull()]

In [22]:
result_prop_served = {}

for key in model.var_prop_bg_at_site.keys():
    if prop_bg_at_site_df.loc[key]:
        result_prop_served[key] = model.var_prop_bg_at_site[key].value

In [23]:
unique_hubs = []

for key in result_prop_served.keys():
    if round(result_prop_served[key], 3) > 0:
            unique_hubs.append(key[1])

set(unique_hubs)

{'166435596', '31273323', '32935760', '438203854', '943344307'}

In [24]:
result_prop_served

{('G06001303310001', '31273323'): 1.0,
 ('G06001303310002', '31273323'): 1.0,
 ('G06001303320001', '31273323'): 1.0,
 ('G06001303320002', '31273323'): 1.0,
 ('G06001303320005', '31273323'): 1.0,
 ('G06001303320006', '31273323'): 1.0,
 ('G06001303320007', '31273323'): 1.0,
 ('G06001303551151', '438203854'): 0.834398198753707,
 ('G06001303552001', '31273323'): 0.192279297741632,
 ('G06001303660012', '943344307'): 1.0,
 ('G06001303660013', '943344307'): 1.0,
 ('G06001303660021', '943344307'): 1.0,
 ('G06001303660023', '943344307'): 1.0,
 ('G06001303671003', '943344307'): 1.0,
 ('G06001303671004', '943344307'): 1.0,
 ('G06001303672002', '943344307'): 1.0,
 ('G06001303680011', '166435596'): 1.0,
 ('G06001303680012', '166435596'): 1.0,
 ('G06001303680013', '166435596'): 1.0,
 ('G06001303680014', '166435596'): 1.0,
 ('G06001303690013', '166435596'): 1.0,
 ('G06001303690014', '166435596'): 0.487599533558776,
 ('G06001303690014', '943344307'): 0.512400466441224,
 ('G06001303720002', '166435596'